In [ ]:
%cd /content

repo_name = "VoxCPM-ja"
repo_url = "https://github.com/q9uri/VoxCPM-ja.git"
import os

if not os.path.exists(repo_name):
    !git clone {repo_url}
else:
    print(f"{repo_name} already exists. Pulling latest changes...")

%cd {repo_name}
!git pull

In [ ]:
with open("pyproject.toml", "r") as f:
    text = f.read()

text = text.replace('"jpreprocess",', "")

with open("pyproject.toml", "w") as f:
    text = f.write(text)

!pip install -e .

In [ ]:
#patch g2p (colab and hf space don't install jpreprocess on 3.12)

!pip install pyopenjtalk-plus

text = """
import pyopenjtalk
from .lib.ja.voicevox_g2p.kana_converter import create_kana
from .lib.ja.voicevox_g2p.text_analyzer import full_context_labels_to_accent_phrases

def g2p(norm_text) -> str:
    full_context_labels = pyopenjtalk.extract_fullcontext(norm_text)
    accent_phrases = full_context_labels_to_accent_phrases(full_context_labels)
    kana = create_kana(accent_phrases)
    return kana
"""

with open("src/voxcpm/text/japanese.py", mode="w") as f:
  f.write(text)



In [ ]:
#download lora model
from huggingface_hub import hf_hub_download
import os

repo_id = "WariHima/1.5-large-ja-rev-a"
filename = ["lora_config.json", "lora_weights.safetensors", "optimizer.pth", "scheduler.pth"]

for file in filename:
  path = hf_hub_download(
      repo_id=repo_id,
      filename=file,
      local_dir="./checkpoints/1.5-large-ja-rev-a",
      local_dir_use_symlinks=False
  )
print(f"Downloaded to: {path}")

In [ ]:
#download asr model
from huggingface_hub import hf_hub_download
import os

repo_id = "FunAudioLLM/SenseVoiceSmall"
filename = ["am.mvn", "chn_jpn_yue_eng_ko_spectok.bpe.model", "config.yaml", "configuration.json", "model.pt"]

for file in filename:
  path = hf_hub_download(
      repo_id=repo_id,
      filename=file,
      local_dir="./checkpoints/SenseVoiceSmall",
      local_dir_use_symlinks=False
  )
print(f"Downloaded to: {path}")

In [ ]:
with open("app.py", "r") as f:
    text = f.read()

# share=True の置換
replace_list = [
    "interface.queue(max_size=10, default_concurrency_limit=1).launch(server_name=server_name, server_port=server_port, show_error=show_error)",
    "interface.queue(max_size=10, default_concurrency_limit=1).launch(server_name=server_name, server_port=server_port, show_error=show_error, share=True)",
]
text = text.replace(replace_list[0], replace_list[1])

with open("app.py", "w") as f:
    f.write(text)

#run gradio webui

!python app.py

In [ ]:
#option
#download korean lora model
from huggingface_hub import hf_hub_download
import os

repo_id = "WariHima/1.5-large-ko-rev-a"
filename = ["lora_config.json",
            "lora_weights.safetensors",
            "optimizer.pth",
            "scheduler.pth",
            "KoichiYasuoka__roberta-large-korean-morph-upos/config.json",
            "KoichiYasuoka__roberta-large-korean-morph-upos/esupar.model",
            "KoichiYasuoka__roberta-large-korean-morph-upos/pytorch_model.bin",
            "KoichiYasuoka__roberta-large-korean-morph-upos/special_tokens_map.json",
            "KoichiYasuoka__roberta-large-korean-morph-upos/tokenizer.json",
            "KoichiYasuoka__roberta-large-korean-morph-upos/tokenizer_config.json",
            "KoichiYasuoka__roberta-large-korean-morph-upos/vocab.txt",
            ]

for file in filename:
  path = hf_hub_download(
      repo_id=repo_id,
      filename=file,
      local_dir="./checkpoints/1.5-large-ko-rev-a",
      local_dir_use_symlinks=False
  )


print(f"Downloaded to: {path}")

In [ ]:
# patch for colab in korean
target_path = "src/voxcpm/text/lib/ko/g2pk4/normalaizer/japanese.py"
with open(target_path, "r") as f:
  text = f.read()

text = text.replace("from ....ja.voicevox_g2p import pyopenjtalk", "import pyopenjtalk")

with open(target_path, "w") as f:
    f.write(text)


In [ ]:
# korean language demo!
# g2p周りのエラー出るけど動くよ！
with open("app.py", "r") as f:
    text = f.read()

# share=True の置換
replace_list = [
    "1.5-large-ja-rev-a",
    "1.5-large-ko-rev-a",
]
text = text.replace(replace_list[0], replace_list[1])

with open("app.py", "w") as f:
    f.write(text)

#run gradio webui

!python app.py